# STA130 TUT 12 (Nov22)<br><br> ⛅ 🌤️ ☀️ <u>Metrics + Ethics = Methrics<br>more *Metrics* but also now *Ethics*<u>


## ♻️ 📚 Review/Questions [15 minutes]
1. Questions from the Nov15 TUT or the Nov18 LEC?
    1. The **confusion matrix** (and how you get it) and the difference between **out of sample** versus **in sample predictive performance** are the most relevant concepts to review for the material of the upcoming week 
    
> If anything is unclear about the concepts and methodologies we've covered so far, now is a quick opportunity to raise your questions before we dive into the ethical considerations that naturally appear next!

## 🚧 🏗️ Demo: Lyme Disease and Alternative Metrics [25 minutes]
<!-- *Biases and Unfairness*? <br>Clinical Trials and the 'Adult Income' Dataset -->


> Lyme disease is a tick-borne illness that can cause severe health problems if not treated early. Early symptoms are often nonspecific and can mimic less severe viral illnesses, making accurate diagnosis necessary. A healthtech company has developed a predictive machine learning model intended to assist doctors in diagnosing Lyme disease based on symptoms, history of tick exposure, and initial blood tests. The model was tested in a clinical trial involving patients suspected of having Lyme disease based on their symptoms and exposure history. 




### A. *Confusion matrix* and the standard *metrics* 

|                 | Predict Disease | Predict No Disease |
|-----------------|-----------------|--------------------|
| **Disease**     | 23 (TP)         | 7 (FN)             |
| **No Disease**  | 90 (FP)         | 280 (TN)           |

- **True Positives (TP)**: 23 - The model correctly predicted the disease.
- **False Negatives (FN)**: 7 - The model incorrectly predicted no disease when the disease was present.
- **False Positives (FP)**: 90 - The model incorrectly predicted the disease when it was absent.
- **True Negatives (TN)**: 280 - The model correctly predicted no disease.

**Accuracy**: $\frac{TP + TN}{TP + FN + FP + TN} = \frac{23 + 280}{23 + 7 + 90 + 280} = 0.759$

**Specificity**: $\frac{TN}{FP + TN} = \frac{280}{90 + 280} = 0.757$

**Sensitivity** : $\frac{TP}{TP + FN} = \frac{23}{23 + 7} = 0.767$

### B. Alternative *metrics* that might give a clearer and more direct picture of what's happening

The **metrics** above measure *correctness*, but perhaps we should first consider **metrics** that are instead focused on *incorrectness*. 

**False Positive Rate (1 minus specificity)**: $1 - \frac{TN}{FP + TN} = 1 - \frac{280}{90 + 280} = 1 - 0.757 = 0.243$

Additionally, all of the above **metrics** are measured with respect to the "truth" *which we don't know in practice (and only know for this observed data)*. So we could also instead consider **metrics** which measure performance relative to the prediction that we make by considering a "different denominator".

**Precision**: $\frac{TP}{TP + FP} = \frac{23}{23 +90} = 0.204$


### C. What does *Class Imbalance* do here?

There are 30 cases of Lyme Disease and 270 individuals that do not have Lyme Disease.
Since only 7.5% of the people here have Lyme Disease what if we just predict no Lyme Disease for everyone? 

|                 | Predict Disease | Predict No Disease |
|-----------------|-----------------|--------------------|
| **Disease**     | 0 (TP)          | 30 (FN)            |
| **No Disease**  | 0 (FP)          | 370 (TN)           |

**Accuracy**: $\frac{TP + TN}{TP + FN + FP + TN} = \frac{0 + 370}{0 + 30 + 0 + 370} = 0.90$

Compare this against the original **metrics** and consider how **Class Imbalance** seems to warp our view of performance.  

**Sensitivity** : $\frac{TP}{TP + FN} = \frac{23}{23 + 7} = 0.767$

**Precision**: $\frac{TP}{TP + FP} = \frac{23}{23 +90} = 0.204$


## 💬 🗣️ Communication Activity \#1 [30 minutes]:<br>*Ethical Considerations* for  different decision-making perspectives 

_Break up into **project team groups** and then pair up **project team groups** into **three pairs** of **project team groups**. Alernatingly divide up the **30-minutes** into **five-minutes** of discussion **project team group pairs** followed by **five-minutes** of discussion between everyone._ ALL three **project team group pairs** discuss and answer ALL questions below, but ideally each of the **three pairs** should take the lead on facilitating the full group discussions at minutes "5", "15", and "25". 

Discuss and answer the following questions supposing all stakeholder parties involved have reviewed and understand the **confusion matrix** and **metric performance** measures above.


### 1. *Patient Perspective: Should I Trust Medical Advice based on this Diagnostic Test?*

> Here are two more **metrics** to consider relative to the orignal **confusion matrix**.
> 
> **False Negative Rate (1 minus sensitivity)**: 
> 
> $1- \frac{TP}{TP + FN} = 1- \frac{23}{23 + 7} = 0.232 \quad \longleftarrow \quad$ "a pretty high rate of truly positive cases are predicted negative" 
> 
> **Negative Predictive Value** (the "negative prediction" version of **Precision**): 
> 
> $\frac{TN}{FN + TN} = \frac{280}{7 + 280} = 0.976 \quad \longleftarrow \quad$ "most negative predictions are really negative" 

1. **Context**: A patient feels very sick and a Lyme disease test comes back "negative"/"positive".  
    
2. **Question**: How should the patient approach medical advice based on the outcome of the test?
    
3. **Follow up**: How should the patient view follow-up testing relative to the test result they got?


### 2. *Healthcare Provider Perspective: What next steps should be recommend now*?

1. **Context**: A Lyme disease test has come back "negative"/"positive" for a sick patient.

2. **Question**: How should the risks of possible treatment side-effects be balanced against the potential untreated progression of Lyme disease?
    
3. **Follow up**:  What steps could be taken to manage patient anxiety or mistrust towards any diagnostic result or medical treatment recommendation? 
    

### 3. *Healthcare System Perspective: Model Use and Calibration*

1. **Context**: Administrators assess the model's impact on healthcare practices.

2. **Question**: How should the relatively high rates of **false positives** and **false negatives** be handled? 

3. **Follow up**: Should the healthcare system recalibrate the model to predict more *postives* or *negatives* in order to tradeoff the **sensitivity** and **specificity** rates based on the current performance metrics? 

> Consider the implications of the following recalibration on patient care, resource workflow allocation, and ethical responsibilities of healthcare providers.
 
|                 | Predict Disease | Predict No Disease |
|-----------------|-----------------|--------------------|
| **Disease**     | 30 (TP)         | 0 (FN)             |
| **No Disease**  | 150 (FP)        | 260 (TN)           |


## 📢 👂 Communication Activity \#2 [30 minutes]

_Use the same **project team group pairs** from above as follows. Select **TWO** of the **three pairs** to be debate pairs and split the remaining **project team group pair** to be the discussion moderator-judges for the two debates._  

- For each ethical scenario debate the moderator-judges assign one group to defend a position and the other to oppose it. Groups are allowed 8-minutes to discuss and prepare their arguments, and then each debate will last 11-minutes using the following format.

- TWO different group members from the "defending group" will each present ONE argument for each of the "Ethical Concerns" below (with a 2-minute limit per group); then, the "opposing group" similarly argues TWO points. The remaining members from each group who have not yet argued may then provide rebuttals (with a 2-minute limit per group), starting with the "opposing team" and then concluding with the "defending team" (in a "sandwich" format). There will then be 3-minutes for the moderator-judges to reflect on the arguments and vote on the following two points. 
    - The effectiveness of the debating teams in presenting their points 
    - The position of the debating teams which students personally agree with

### Scenario 1: Healthcare AI for Predicting Patient Risks
**Background**: A healthcare startup has developed an AI system that uses patient data such as medical history, lifestyle choices, and genetic information to predict individuals’ risks of developing chronic diseases.

- **Group A: defending the use of the AI system for healthcare**. 
- **Group B: opposing the use of the AI system for healthcare**. 

### Scenario 2: Predictive Policing
**Background**: A city's police department has implemented a new AI system designed to predict crime hotspots by analyzing historical crime data and demographic information. The system is intended to optimize police patrols and prevent crimes before they occur.

- **Group A: defending the use of the AI system for policing**. 
- **Group B: opposing the use of the AI system for policing**. 


<!--
### Scenario 3: AI in education
**Background**: A statistics class has incorporated AI into their learning environment.

- **Group A: defending the use of the AI system in a classroom**. 
- **Group B: opposing the use of the AI system in a classroom**. 
-->